# Creation of a minimal RBA model using RBApy's xml module.

In this section, we'll create a cell that magically transforms a carbon source into biomass. This example will walk us through the basic elements of an RBA model, as well as RBApy's interface for creating/modifying models.

## Creation of an empty model with RBApy

Let's start by loading the RBApy package and creating an empty RBA model:

In [1]:
import rba

my_model = rba.RbaModel()

The model contains all the XML substructures, but nothing is instantiated yet:

In [2]:
print(vars(my_model))
print(str(my_model.metabolism.compartments) + ' has length ' + str(len(my_model.metabolism.compartments)))

{'_constraint_matrix': None, 'output_dir': '', 'enzymes': <rba.xml.enzymes.RbaEnzymes object at 0x10cd85cf8>, 'rnas': <rba.xml.macromolecules.RbaMacromolecules object at 0x10cd70128>, 'proteins': <rba.xml.macromolecules.RbaMacromolecules object at 0x10b3d09e8>, 'targets': <rba.xml.targets.RbaTargets object at 0x180de825c0>, 'parameters': <rba.xml.parameters.RbaParameters object at 0x103e548d0>, 'dna': <rba.xml.macromolecules.RbaMacromolecules object at 0x180de6ea90>, 'density': <rba.xml.density.RbaDensity object at 0x103e54080>, 'processes': <rba.xml.processes.RbaProcesses object at 0x180de6ec18>, 'medium': {}, 'metabolism': <rba.xml.metabolism.RbaMetabolism object at 0x103e39ef0>}
<rba.xml.metabolism.ListOfCompartments object at 0x103dcdac8> has length 0


We can write our empty model to files, then reload them:

In [3]:
import os

if not os.path.exists("minimal_example"):
    os.mkdir('minimal_example')
my_model.write('minimal_example/', )
my_model = rba.RbaModel.from_xml('minimal_example')
%less minimal_example/metabolism.xml

## metabolism.xml: compartments, metabolites and reactions

We'll start by populating the Metabolism substructure, which contains compartments, metabolites and metabolic reactions.

In our minimal cell, we will consider three metabolites: a carbon source, a protein component precursor (used to build proteins) and a vital metabolite (used to accumulate biomass). The carbon source is present in two forms: extracellular carbon source, and intracellular carbon source.

Let's add these metabolites to the model!

In [4]:
my_model.metabolism.species.append(rba.xml.Species('M_carbon_source_e', boundary_condition = True))
my_model.metabolism.species.append(rba.xml.Species('M_carbon_source_c', boundary_condition = False))
my_model.metabolism.species.append(rba.xml.Species('M_protein_component_precursor_c', boundary_condition = False))
my_model.metabolism.species.append(rba.xml.Species('M_biomass_c', boundary_condition = False))                                   

Every time we want to create an XML element, we access its class in the `rba.xml` subpackage, then append it to the list where it belongs.

We use the following conventions: every metabolite id starts with `M_` and ends with a suffix that indicates its compartment, e.g. `_e` is the extracellular space. `boundary_condition` is set to `True` for extracellular species, indicating that their concentration is determined by the medium composition.

We can check that the list of species has been updated:

In [5]:
print(len(my_model.metabolism.species))
print(my_model.metabolism.species[0].id)
my_model.write()
%less minimal_example/metabolism.xml

4
M_carbon_source_e


Note that this time, we do not need to specify where to write the model, it will automatically override the last location where we wrote it.
Let's continue by adding the compartments: the extracellular space and the cytosol.

In [6]:
my_model.metabolism.compartments.append(rba.xml.Compartment('extracellular'))
my_model.metabolism.compartments.append(rba.xml.Compartment('cytosol'))
my_model.write()
%less minimal_example/metabolism.xml

Finally let's create some reactions:

In [7]:
reaction_1 = rba.xml.Reaction('R_transport', reversible = False)
reaction_1.reactants.append(rba.xml.SpeciesReference('M_carbon_source_e', 1))
reaction_1.products.append(rba.xml.SpeciesReference('M_carbon_source_c', 1))
reaction_2 = rba.xml.Reaction('R_protein_component_precursor', reversible = False)
reaction_2.reactants.append(rba.xml.SpeciesReference('M_carbon_source_c', 1))
reaction_2.products.append(rba.xml.SpeciesReference('M_protein_component_precursor_c', 1))
reaction_3 = rba.xml.Reaction('R_biomass', reversible = False)
reaction_3.reactants.append(rba.xml.SpeciesReference('M_carbon_source_c', 1))
reaction_3.products.append(rba.xml.SpeciesReference('M_biomass_c', 1))
my_model.metabolism.reactions.append(reaction_1)
my_model.metabolism.reactions.append(reaction_2)
my_model.metabolism.reactions.append(reaction_3)
my_model.write()
%less minimal_example/metabolism.xml

...and add the medium concentration for our extracellular species:

In [8]:
my_model.medium['M_carbon_source'] = 10
my_model.write()
%less minimal_example/medium.tsv

Here we go, we have successfully created a cell that is able to import a carbon source and create biomass from it! We can already go ahead and solve it:

In [9]:
results = my_model.solve()
results.mu_opt

2.5

We obtain infinite growth rate (growth rate is capped at 2.5)! Indeed, there is no constraint on our reactions so far, our cell is theoretically able to take up the carbon source and transform it to biomass at an infinite rate (hence the warnings)! We need enzymes in our model! But before we can define enzymes, we need to define proteins!

## proteins.xml, rnas.xml, dna.xml: macromolecules

The model contains 3 macromolecule files: one for proteins, one for RNAs, one for DNA. In this example, we'll ignore RNAs and DNA, but note that the format of the 3 files is strictly the same, so proteins will cover it all.

In an RBA model, a macromolecule is an assembly of components: proteins are composed of amino acid residues (and vitamins and ions), RNAs and DNA out of nucleotides. How these components were assembled from metabolites will be covered at a later stage, when we talk about processes. Currently, all we want to provide is a static descriptions of the composition of macromolecules in the system.

We will include two proteins in our model, and say that they are composed out of protein component residues. Let's start with the definition of components:

In [10]:
my_model.proteins.components.append(rba.xml.Component(id_='protein_component_residue', name='Protein residue', type_='residue', weight=1))

`name` and `type` are two labelling attributes that can be used for human readability, but they have no influence on the simulations. `weight` is more important: it indicates how much volume is taken up by one component. The unit is arbitrary: here we will consider that the protein residue defines the unit volume.

Now let's define two proteins as a sum of components:

In [11]:
protein_1 = rba.xml.Macromolecule('small_protein', 'cytosol')
protein_1.composition.append(rba.xml.ComponentReference('protein_component_residue', 10))
my_model.proteins.macromolecules.append(protein_1)
protein_2 = rba.xml.Macromolecule('large_protein', 'cytosol')
protein_2.composition.append(rba.xml.ComponentReference('protein_component_residue', 20))
my_model.proteins.macromolecules.append(protein_2)

Our two proteins are located in the cytosol (one of the compartments we defined earlier). The small proteins contains 10 residues (volume 10), the large protein contains 20 residues (volume 20).

That's all we need for our proteins, let's wrap it up:

In [12]:
my_model.write()
%less minimal_example/proteins.xml

## enzymes.xml: catalytic constraints

We can finally build our enzymes: we have reactions that await to be constrained, and proteins to serve as building blocks for our enzymes!

Let's start by adding catalytic constants for our transport reaction:

In [13]:
enzyme_1 = rba.xml.Enzyme(id_='R_transport_enzyme', reaction='R_transport', forward_efficiency='kcat_transport', backward_efficiency='zero')
enzyme_1.machinery_composition.reactants.append(rba.xml.SpeciesReference('large_protein', 2))
my_model.enzymes.enzymes.append(enzyme_1)

We just defined an enzyme composed of 2 large proteins, catalyzing the reaction `R_transport`, with an efficiency given by parameters `kcat_transport` and `zero`. Note that efficiences *cannot* be numerical values, they must be parameter ids. What are parameters? Oh right, we haven't defined parameters yet, but we will do so in the next session!

Let's add enzymes for our other reactions, then update our model:

In [14]:
enzyme_2 = rba.xml.Enzyme(id_='R_protein_precursor_enzyme', reaction='R_protein_component_precursor', forward_efficiency='kcat_precursor', backward_efficiency='zero')
enzyme_2.machinery_composition.reactants.append(rba.xml.SpeciesReference('small_protein', 2))
my_model.enzymes.enzymes.append(enzyme_2)
enzyme_3 = rba.xml.Enzyme(id_='R_biomass_enzyme', reaction='R_biomass', forward_efficiency='kcat_biomass', backward_efficiency='zero')
enzyme_3.machinery_composition.reactants.append(rba.xml.SpeciesReference('small_protein', 1))
enzyme_3.machinery_composition.reactants.append(rba.xml.SpeciesReference('large_protein', 1))
my_model.enzymes.enzymes.append(enzyme_3)

In [15]:
my_model.write()
%less minimal_example/enzymes.xml

## parameters.xml: centralizing all model parameters

The only file where numerical values are allowed is the parameter file! In other files, whenever you want to define an efficiency, a flux, a concentration, you must provide a parameter identifier. All parameters are centralized: if you want to keep the same system but play around with parameter values, this is the only file you need to modify!

By default, a parameter is seen as a function of growth rate. There are a limited number of function types that you may define: 'constant', 'linear', 'exponential', 'michaelisMenten' and 'inverse'. Every function has its own parameters (see document with complete description of the XML format for more details.). Let's define our catalytic activities using constants:

In [16]:
parameter_1 = rba.xml.Function('kcat_transport_base', 'constant', {'CONSTANT': 10})
my_model.parameters.functions.append(parameter_1)
parameter_2 = rba.xml.Function('kcat_precursor', 'constant', {'CONSTANT': 10})
my_model.parameters.functions.append(parameter_2)
parameter_3 = rba.xml.Function('kcat_biomass', 'constant', {'CONSTANT': 10})
my_model.parameters.functions.append(parameter_3)
parameter_4 = rba.xml.Function('zero', 'constant', {'CONSTANT': 0})
my_model.parameters.functions.append(parameter_4)

Constant functions have a single parameter called `CONSTANT`, that's all we need here!

Now what if we would like our transport efficiency to depend on the concentration of our extracellular carbon source? Function variables may also be defined as medium concentrations. Earlier, we defined a medium concentration 'M_carbon_source' (with value 10). We can reuse this id to define a function:

In [17]:
parameter_4 = rba.xml.Function('transport_factor', 'michaelisMenten', {'kmax': 1, 'Km': 0.5}, variable='M_carbon_source_e')
my_model.parameters.functions.append(parameter_4)

But wait! In the previous section, we said that the transporter had forward efficiency `kcat_transport`. Instead, we defined two parameters: `kcat_transport_base`, the base efficiency, and `transport_factor`, a factor that modulates the base efficiency based on the concentration of the metabolite that's imported!

In that case, we need to use an Aggregate to combine the two parameters: we obtain the overall transporter efficiency by multiplying the base efficiency by the transport factor:

In [18]:
aggregate_1 = rba.xml.Aggregate('kcat_transport', type_='multiplication')
aggregate_1.function_references.append(rba.xml.FunctionReference('kcat_transport_base'))
aggregate_1.function_references.append(rba.xml.FunctionReference('transport_factor'))
my_model.parameters.aggregates.append(aggregate_1)

We have defined all the parameters we needed so far. Let's take a look at the result:

In [19]:
my_model.write()
%less minimal_example/parameters.xml

Let's go even further and try to solve our model again!

In [20]:
results = my_model.solve()
results.mu_opt

2.5

Hmm we got rid off the warnings about the missing enzymes, but the growth rate is still infinite... What's going on? The constaints we added specified how many enzymes are needed to sustain a given flux. For example, because kcat is 10, in order to produce a flux of 1 mmol/gCDW/h (????) of `R_biomass`, we need 0.1 mmol of `R_biomass_enzyme`. However, nothing prevents our cell from producing an infinite number of enzymes, achieving infinite fluxes!

Remember how enzymes are built out of proteins, proteins out of components and components have some basic volume? We will use this idea to add a constraint on space: a cell should only produce as many enzymes as can be contained in the cytosol!

## density.xml: density constraints for compartments

For our minimal model, we will consider that the cytosol can contain at most 10,000 protein residues!

In [21]:
density_constraint = rba.xml.TargetDensity('cytosol')
density_constraint.upper_bound = 'maximal_cytosol_density'
my_model.density.target_densities.append(density_constraint)

A density constraint has 3 attributes: lower_bound, upper_bound and value. The nature of the constraint depends on the attributes that you define. Here we simply specify an upper bound, meaning that the cytosol may contain from 0 to 10 residues. We could also define a lower bound, or specify a fixed value that must be reached.

Remember that all numeric values must be defined in the parameter file? Let's add our new parameter right now:

In [22]:
my_model.parameters.functions.append(rba.xml.Function('maximal_cytosol_density', 'constant', {'CONSTANT': 10}))

Let's take a quick look at our density file:

In [23]:
my_model.write()
%less minimal_example/density.xml

... and solve our mode again!

In [24]:
results = my_model.solve()
results.mu_opt

2.5

Hang on a second! The growth rate is still infinite? If you are familiar with FBA, you may notice that something essential is missing in our model: requirements for biomass production. There is currently no production requirement! Our cell is telling us that if it doesn't have to produce anything to grow, it can grow at infinite growth rate!

## targets.xml: production requirements

Even if for us humans, the `R_biomass` reaction has a transparent name, we need to tell the computer model that this is what it needs to produce in order to grow. In our model, biomass is a generic term that encompasses everything a cell needs to produce in order to divide: cell membranes, cell wall, DNA, etc. We will specify this requirement by indicating that these elements need to be kept at a given concentration for the cell to be viable:

In [25]:
new_target = rba.xml.TargetSpecies('M_biomass_c')
new_target.value = 'target_biomass_production'
new_target_group = rba.xml.TargetGroup('biomass_production')
new_target_group.concentrations.append(new_target)
my_model.targets.target_groups.append(new_target_group)

Every production requirement must be part of a target group. This group has no meaning from a computational point of view, it is used to gather requirements that are related. A target can be a species or a reaction. In the case of a reaction, the requirement must be a flux, in the case of a species, it may be a flux or a concentration (see XML specification document for more details).

Let's add the parameter that corresponds to our production requirement:

In [26]:
my_model.parameters.functions.append(rba.xml.Function('target_biomass_production', 'constant', {'CONSTANT': 1}))

Finally, let's take a look at our targets file

In [27]:
my_model.write()
%less minimal_example/targets.xml

... and solve the model!

In [28]:
results = my_model.solve()
results.mu_opt

1.3888883590698242

We did it! Now we can play with the maximal density parameter and check how it influences the estimated growth rate!

In [29]:
maximal_density_fn = my_model.parameters.functions.get_by_id('maximal_cytosol_density')
maximal_density = maximal_density_fn.parameters.get_by_id('CONSTANT')
maximal_density.value = 10
results = my_model.solve()
print(results.mu_opt)
maximal_density.value = 1
results = my_model.solve()
print(results.mu_opt)

1.3888883590698242
0.13888835906982422


Even though we have escaped infinite growth rate, our model is still missing a very important part. So far, we have constrained our cell to produce biomass to grow. In order to produce biomass, the cell must use its metabolic reactions. In order to use these reactions, it needs to produce enzymes. In order to produce enzymes, it needs to produce proteins. However, we have not told it how proteins are made!

Currently, our cell only knows how to compute the proteins' volume, which is enough to keep growth rate finite, but it does not know how to assemble proteins from metabolites. By default, it assumes that there is nothing to assemble: our proteins (and hence our enzymes) appear out of thin air, at no cost!

This situation is remedied by Processes, which can be seen as a meta-reaction that specifies how macromolecule components are built out of metabolites.

## processes.xml: macromolecule production

We will create a process that tells our cell how it should build the protein's components, which we called `protein_component_residue` earlier. In a realistic model, we would have to specify how every amino acid residue is assembled from charged tRNAs and GTP, releasing uncharged tRNAs and GDP in the process. For our simplified example, we will simply assume that a `protein_component_residue` is obtained by combining a `protein_component_precursor` and a `carbon_source` (for energy). We will also assume that a `carbon_source` is necessary to initiate translation (for energy).

These component production reactions are stored in a ProcessingMap:

In [30]:
new_map = rba.xml.ProcessingMap(id_='translation_map')
new_map.constant_processing.reactants.append(rba.xml.SpeciesReference('M_carbon_source_c', 1))
residue_processing = rba.xml.ComponentProcessing(component='protein_component_residue')
residue_processing.reactants.append(rba.xml.SpeciesReference('M_protein_component_precursor_c', 1))
residue_processing.reactants.append(rba.xml.SpeciesReference('M_carbon_source_c', 1))
new_map.component_processings.append(residue_processing)
my_model.processes.processing_maps.append(new_map)

We need to define one ComponentProcessing per component. Because we only have one component, our job is done! Let's take a look at the updated processes file:

In [31]:
my_model.write()
%less minimal_example/processes.xml

Now let's create a translation process based on this ProcessingMap:

In [32]:
translation = rba.xml.Process(id_='translation', name='Translation process')
processing = rba.xml.Processing(map_='translation_map', set_='protein')
for protein in my_model.proteins.macromolecules:
    processing.inputs.append(rba.xml.SpeciesReference(protein.id, 1))
translation.processings.productions.append(processing)
my_model.processes.processes.append(translation)

A process can define how macromolecules are produced or degraded. The Processing structure is used to select macromolecules that enter the process (through the set - protein, rna or, dna - and ids within that set) and how they are assembled (resp. degraded) component-wise by pointing to a ProcessingMap.

Note that a process can theoretically contain several sets of input and ProcessingMaps. Conversely, a macromolecule can be listed as input of several processes. In the end, the production (resp. degradation) reaction of a macromolecule is obtained by combining all the ProcessingMaps of the processes it is an input of.

In [33]:
my_model.write()
%less minimal_example/processes.xml

Let's see how that change affected growth rate!

In [34]:
results = my_model.solve()
results.mu_opt

0.05603671073913574

That's a huge drop in growth rate! Building enzymes is extremely costly! However, we can make this even worse! So far we have implicitly assumed that proteins assemble spontaneously: there is no ribosome!

We can add a molecular machinery to a process through the Machinery stucture:

In [35]:
ribosome = rba.xml.Machinery()
ribosome.machinery_composition.reactants.append(rba.xml.SpeciesReference('small_protein', 1))
ribosome.machinery_composition.reactants.append(rba.xml.SpeciesReference('large_protein', 1))
my_model.processes.processes.get_by_id('translation').machinery = ribosome

We successfully created our ribosome, but we still need to specify how much it is needed to assemble individual components:

In [36]:
translation_map = my_model.processes.processing_maps.get_by_id('translation_map')
translation_map.component_processings[0].machinery_cost = 0
ribosome.capacity.value = 'ribosome_capacity'
my_model.parameters.functions.append(rba.xml.Function('ribosome_capacity', 'constant', {'CONSTANT': 100}))


The capacity of a process is a parameter that is closely related to enzyme efficiency. It relates to the number of components a machinery can process per unit of time. The exact interpretation of the capacity depends on the `machinery_cost` associated with the components. Because in our example, the `machinery_cost` for assembling a component is 1, our ribosome has the capacity to assemble up to 100 components per unit of time.

Similar to density constraints and targets, capacity has 3 attributes `value`, `lower_bound` and `upper_bound`. We can specify whether our ribosome can process up to, at least, or exactly 100 component per second.

We finished implementing our process and our minimal example, let's look at the final processes file:

In [37]:
my_model.write()
%less minimal_example/processes.xml

We can solve the full model:

In [38]:
results = my_model.solve()
results.mu_opt

0.05603671073913574

## Parameters.xml: overview of the impact of parameters

In this section, we will go back to the parameter file and quickly study how key parameters affect the outcome of the model.

In [39]:
%less minimal_example/parameters.xml
maximal_density = my_model.parameters.functions.get_by_id('maximal_cytosol_density').parameters.get_by_id('CONSTANT')
transport_kcat = my_model.parameters.functions.get_by_id('kcat_transport_base').parameters.get_by_id('CONSTANT')
precursor_kcat = my_model.parameters.functions.get_by_id('kcat_precursor').parameters.get_by_id('CONSTANT')
biomass_kcat = my_model.parameters.functions.get_by_id('kcat_biomass').parameters.get_by_id('CONSTANT')
ribosome_capacity = my_model.parameters.functions.get_by_id('ribosome_capacity').parameters.get_by_id('CONSTANT')

The growth rate is either limited by the density constraints or the catalytic constraints. If we keep increasing the maximal density, the solution remains bounded:

In [40]:
maximal_density.value = 10
results = my_model.solve()
print(results.mu_opt)
maximal_density.value = 100
results = my_model.solve()
print(results.mu_opt)
maximal_density.value = 1000
results = my_model.solve()
print(results.mu_opt)
maximal_density.value = 10000
results = my_model.solve()
print(results.mu_opt)

0.08795976638793945
0.09327113628387451
0.09383797645568848
0.09389519691467285


Roughly speaking, there is a point where importing the carbon source is not worth it anymore because it is entirely consumed to make new enzymes. We can increase the growth rate by increasing enzyme efficiencies:

In [41]:
transport_kcat.value = 100
precursor_kcat.value = 100
biomass_kcat.value = 100
results = my_model.solve()
print(results.mu_opt)

0.938953161239624


By increasing the capacity of enzymes, the cell produces them in smaller concentrations, which means more energy for biomass. In our example, the system is strongly limited by the transport reaction:

In [42]:
precursor_kcat.value = 100000
biomass_kcat.value = 100000
results = my_model.solve()
print(results.mu_opt)
precursor_kcat.value = 1000000
biomass_kcat.value = 1000000
results = my_model.solve()
print(results.mu_opt)

1.161106824874878
1.1613553762435913


Modifying the efficiency of the limiting enzyme dramatically affects the growth rate:

In [43]:
transport_kcat.value = 100
results = my_model.solve()
print(results.mu_opt)
transport_kcat.value = 200
results = my_model.solve()
print(results.mu_opt)
transport_kcat.value = 50
results = my_model.solve()
print(results.mu_opt)

1.1613553762435913
2.322656512260437
0.5806845426559448


As mentioned earlier, there is a point where density constraints become stronger than catalytic constraints:

In [44]:
maximal_density.value = 10000
results = my_model.solve()
print(results.mu_opt)
maximal_density.value = 100
results = my_model.solve()
print(results.mu_opt)
maximal_density.value = 10
results = my_model.solve()
print(results.mu_opt)
maximal_density.value = 1
results = my_model.solve()
print(results.mu_opt)

0.5806845426559448
0.577893853187561
0.5537021160125732
0.3903120756149292
